In [ ]:
# importing all the required modules
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 

from keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing import pad_sequences
'''
   Latest tensorflow gives error, so replace upper line by lower one
'''
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
#---Hyperparameters---#
VOCAB_SIZE = 30000
EMBEDDING_DIM = 16
EPOCHS = 15
MAX_LEN = 32
TRUNC_TYPE = 'post'
PADD_TYPE = 'post'
UNK_TOK = '<UNK>'
TRAINING_SIZE = 1000

print('\n Completed\n')

In [ ]:
dataset = pd.read_csv('Spam dataset.csv')
print('dataset: ')

print(dataset.head(), '\n')
print('Null values in the dataset: ')

print(dataset.isnull(), '\n')
print('Sum of Null values in each column: ')

print(dataset.isnull().sum(), '\n')

dataset["Category"] = [1 if each == "spam" else 0 for each in dataset["Category"]]

print('dataset: ')
print(dataset.head(), '\n')

In [ ]:
X = dataset.iloc[:, 1].values
X = X.reshape(-1, 1)
print('Type of X:', type(X))
print('shape of X: ', X.shape)
#print(X)

Y = dataset.iloc[:, 0].values
y = Y.reshape(-1, 1)
print('Type of Y; ', type(Y))
print('shape of Y: ', y.shape)
#print(y)
X = X.tolist()
y = y.tolist()
print('Length of X: ', len(X), '\n')

In [ ]:
#---Train test Split---#
Training_Sentences = X[TRAINING_SIZE:]
Training_Labels = y[TRAINING_SIZE:]
print('Length of training samples ', len(Training_Sentences), '\n')

Testing_Sentences = X[0: TRAINING_SIZE]
Testing_Lables = y[0:TRAINING_SIZE]
print('Length of testing samples', len(Testing_Sentences), '\n')

In [ ]:
#---TOKENIZING THE WORDS---#
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=UNK_TOK)
tokenizer.fit_on_texts(Training_Sentences)

word_index = tokenizer.word_index

#print(word_index, '\n')

Training_Sequences = tokenizer.texts_to_sequences(Training_Sentences)
Training_pad = pad_sequences(Training_Sequences, maxlen=MAX_LEN, padding=PADD_TYPE, truncating=TRUNC_TYPE)

Testing_Sequences = tokenizer.texts_to_sequences(Testing_Sentences)
Testing_pad = pad_sequences(Testing_Sequences, maxlen=MAX_LEN, padding=PADD_TYPE, truncating=TRUNC_TYPE)

In [ ]:
#----BUILDING THE MODEL----#

model = tf.keras.Sequential()
print('-----MODEL INSTANCE Completed-----')
model.add(tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(24, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
print(model.summary())

In [ ]:
#--------Converting this into numpy array--------#
Training_Sequences_padded = np.asarray(Training_pad)
Testing_Sequences_padded = np.asarray(Testing_pad)
Training_Labels = np.asarray(Training_Labels)
Testing_Lables = np.asarray(Testing_Lables)

In [ ]:
#---------Training the MODEL--------#
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(Training_Sequences_padded, Training_Labels, validation_data=(Testing_Sequences_padded, Testing_Lables), epochs=EPOCHS)

In [ ]:
def Plot(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.xlabel("EPOCHS")
    plt.ylabel(string)
    plt.legend([string, 'val_' + string ])
    plt.savefig(string + '.png')
    plt.show()
    
    
Plot(history, "accuracy")
Plot(history, "loss")


print("------Completed------")


In [25]:
text = 'Get 100% off and Rs. 1000000 cashback for this random thing that you won in a lottery!!!!!'
print(text)
Test = tokenizer.texts_to_sequences([text])[0]
Test_padded = pad_sequences([Test], maxlen=MAX_LEN, padding=PADD_TYPE, truncating=TRUNC_TYPE)
Test_padded = np.asarray(Test_padded)
print('Done')
print(Test_padded.shape)

ypred = model.predict(Test_padded)

print('output class: ', ypred)

How are you!!!!!
Done
(1, 32)
1/1 [==============================] - 0s 84ms/step
output class:  [[0.99995685]]


In [26]:
y_pred = model.predict(Testing_Sequences_padded)
print(y_pred.shape)

32/32 [==============================] - 1s 25ms/step
(1000, 1)


In [23]:
print(Testing_Lables.shape)

(1000, 1)


In [27]:
from sklearn.metrics import accuracy_score
# print(accuracy_score(Testing_Lables, y_pred))
''''Classification metrics can't handle a mix of binary and continuous targets''' # so replace the upper line by the lower one
print(accuracy_score(Testing_Lables, y_pred.round(), normalize = False))

882
